In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as BE
from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

# from tensorflow.keras import backend as BE
# from Integrated_Gradients_algorithm import *
# from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config) 

# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

In [2]:
DATA_PATH = '../datasets/ImageNetVal/'
file_name = getfile_name(DATA_PATH)
file_name = np.sort(file_name)

f = open("../datasets/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt",encoding = "utf-8")
val_ground_truth = f.read()
val_ground_truth = val_ground_truth.split('\n')
for i in range(len(val_ground_truth)-1):
    val_ground_truth[i] = int(val_ground_truth[i])

vgg19_json = json.load(open('/public/liujiawei/.keras/models/imagenet_class_index.json','r',encoding="utf-8"))

In [3]:
base_model = VGG19(weights='imagenet')

In [4]:
synsets = scipy.io.loadmat(os.path.join('../datasets', 'ILSVRC2012_devkit_t12', 'data', 'meta.mat'))['synsets']

ILSVRC2012_ID = [s[0][0][0][0] for s in synsets]

index1 = 821
WNID = [s[0][1][0] for s in synsets]
print(WNID[index1])

words = [s[0][2][0] for s in synsets]
print(words[index1])

num_train_images = [s[0][7][0][0] for s in synsets]
print(num_train_images[0])

n04263257
soup bowl
1300


In [5]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [27]:
top_set = 1 # 预测的前n个类中包含真实标签则表示预测正确
total_sample_num = 200
img_num = range(0, total_sample_num)
ground_truth_label = []  # 记录样本的真实标签
predicted_confidence = []  # 记录样本的置信度信息
binary_flag = [] # 记录样本是否被正确分类，正确分类的标记为1，否则标记为0
gradients = [] # 记录样本的梯度信息
sec_gradients = [] # 记录样本的梯度信息
firProb_gradient_sum = [] # 以第一大置信度作为真实标签，计算梯度和
secProb_gradient_sum = [] # 以第二大置信度作为真实标签，计算梯度和
num_class = 1000
DATA_PATH = '../datasets/ImageNetVal/'
file_name = getfile_name(DATA_PATH)
file_name = np.sort(file_name)

pbar = ProgressBar()
for i in pbar(img_num):
    img_path = DATA_PATH + file_name[i]
    img = image.load_img(img_path, target_size=(224, 224))
    x_tmp = image.img_to_array(img)
    x_tmp = np.expand_dims(x_tmp, axis=0)
    x_tmp = preprocess_input(x_tmp)
    pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    
    predicted_confidence.append(pre_tmp)
    ground_truth_label.append(WNID[val_ground_truth[i]-1])
    binary_flag.append(
        get_acc(predict_label = decode_predictions(pre_tmp, top=top_set), 
                               ground_truth=WNID[val_ground_truth[i]-1]
               )
    )
    
#     target_one_hot = np.reshape(to_categorical(np.argmax(pre_tmp), num_class),(1,-1))
#     grads = get_loss_gradients(x_tmp, base_model, target_one_hot)
    grads = get_gradients(x_tmp, base_model, np.argmax(pre_tmp))
    gradients.append(grads)
    firProb_gradient_sum.append(np.sum(np.abs(grads)))
    
    target_label = np.argsort(-pre_tmp) 
    target_label = target_label[:,1][0]
#     target_one_hot = np.reshape(to_categorical(target_label, num_class),(1,-1))
#     grads = get_loss_gradients(x_tmp, base_model, target_one_hot)
    grads = get_gradients(x_tmp, base_model, target_label)
    sec_gradients.append(grads)
    secProb_gradient_sum.append(np.sum(np.abs(grads)))

100% |########################################################################|


In [38]:
Gini = []
pbar = ProgressBar()
for i in pbar(img_num):
    Gini.append(np.sum(np.abs(gradients[i])))
    
#     Gini.append(1 - np.sum(predicted_confidence[i]*predicted_confidence[i]))
    
#     tmp1 = (gradients[i] - sec_gradients[i]) /sec_gradients[i]
#     Gini.append(np.sum(np.abs(tmp1)))
    
#     Gini.append(secProb_gradient_sum[i])

100% |########################################################################|


In [39]:
indexs = np.argsort(Gini)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                   decode_predictions=decode_predictions)
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                    decode_predictions=decode_predictions)
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.688203125
RAUC:  0.8138909495548962


In [53]:
np.save('./featureExtraction/ImageNet_'+str(total_sample_num)+'samplesGradientFeature.npy',{
    'ground_truth_label': np.array(ground_truth_label),
    'predicted_confidence': np.array(predicted_confidence),
    'binary_flag': np.array(binary_flag),
    'gradients': np.array(gradients),
    'firProb_gradient_sum': np.array(firProb_gradient_sum),
    'secProb_gradient_sum': np.array(secProb_gradient_sum),
})